<a href="https://colab.research.google.com/github/iceQHdrop/bioinformatic_data_mining/blob/main/WAM_for_function_finding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WAM_for_function_finding

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
! cp -r drive/MyDrive/Colab_Notebooks/bioinformatic_data_mining/dataset ./

In [87]:
import pandas as pd 
import numpy as np
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


设定路径，读取文件名

In [88]:
train_path = 'dataset/TrainingSet'
test_path = 'dataset/TestingSet'    
train_files = os.listdir(train_path)
test_file = os.listdir(test_path)

构建前景概率矩阵

In [118]:
site_base_count = pd.DataFrame(np.zeros((9, 4)), 
                               columns = ['a', 't', 'c', 'g'],
                               index = [i-3 for i in range(9)],)
whole_seq = ''
site_seqs = []

for train_file in train_files:
    with open(train_path + '/' + train_files[0], 'r') as f:
        text = f.readlines()
        site_positions = re.findall('(\d+)(?=\..)', text[1])    # 提取位置
        seq = ''.join(text[2:]).replace('\n', '').lower()   # 仅保留序列

        for position in site_positions:
            site_seqs.append(seq[int(position) - 4:int(position) + 5])  # 提取位点序列

            for i in range(9):
                base = seq[int(position) - 1 + (i - 3)]   # 统计位点周围9个位置的碱基
                site_base_count.loc[i-3, base] += 1

        whole_seq += seq

matrix_pos = site_base_count/len(site_seqs)
matrix_pos

,a,t,c,g
-3,0.090909,0.272727,0.636364,0.000000
-2,1.000000,0.000000,0.000000,0.000000
-1,0.000000,0.000000,0.000000,1.000000
0,0.181818,0.090909,0.181818,0.545455
1,0.090909,0.727273,0.181818,0.000000
2,0.090909,0.272727,0.181818,0.454545
3,0.272727,0.363636,0.090909,0.272727
4,0.090909,0.363636,0.181818,0.363636
5,0.363636,0.272727,0.090909,0.272727


通过统计序列中所有碱基的出现次数构建背景概率矩阵

In [81]:
p_neg = [whole_seq.count(base)/len(whole_seq) for base in ['a', 't', 'c', 'g']]  
matrix_neg = pd.DataFrame(np.tile(p_neg, (9, 1)),
                          columns = ['a', 't', 'c', 'g'],
                          index = [i-3 for i in range(9)])
matrix_neg

,a,t,c,g
-3,0.227893,0.281851,0.223995,0.266261
-2,0.227893,0.281851,0.223995,0.266261
-1,0.227893,0.281851,0.223995,0.266261
0,0.227893,0.281851,0.223995,0.266261
1,0.227893,0.281851,0.223995,0.266261
2,0.227893,0.281851,0.223995,0.266261
3,0.227893,0.281851,0.223995,0.266261
4,0.227893,0.281851,0.223995,0.266261
5,0.227893,0.281851,0.223995,0.266261


定义$S(X)$


In [120]:
def DecisionFunction(seq):
    score = 0

    for i in range(9):
        pos_score = np.log(matrix_pos.loc[i-3, seq[i]])
        neg_score = np.log(matrix_neg.loc[i-3, seq[i]])
        score += pos_score - neg_score

    return score